#  <center> Problem Set 4 <center>

<center> 7.C01/7.C51 & 20.C01/20.C51 <center>

<b>Name:</b>

<b>Kerberos id:</b>

In [ ]:
########## Simply run this chunk ##########
# Setup
import sys
# !{sys.executable} -m pip install rdkit
# !{sys.executable} -m pip install molvs

!pip install rdkit
!pip install molvs
!pip install wget



In [ ]:
########## Simply run this chunk ##########
# Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from molvs import standardize_smiles
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
import wget

from sklearn import preprocessing
import torch
from torch import nn
from tqdm import tqdm 
from torch import optim
import random

matplotlib.rcParams.update({'font.size': 15})
matplotlib.rc('lines', linewidth=3, color='g')
matplotlib.rcParams['axes.linewidth'] = 2.0
matplotlib.rcParams['axes.linewidth'] = 2.0
matplotlib.rcParams["xtick.major.size"] = 6
matplotlib.rcParams["ytick.major.size"] = 6
matplotlib.rcParams["ytick.major.width"] = 2
matplotlib.rcParams["xtick.major.width"] = 2
matplotlib.rcParams['text.usetex'] = False

_______

## Part 1: Dimensionality Reduction for Molecular Representations

In [ ]:
########## Simply run this chunk ##########
# Download data
path = "https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps4/data/bio_version/jak2.csv"
wget.download(path, "./")

In [ ]:
########## Simply run this chunk ##########
# Load data
jak2 = pd.read_csv("jak2.csv")

In [ ]:
########## Simply run this chunk ##########
# Convert SMILES strings to Morgan fingerprints with rdkit
# Define radius and number of bits for our exercise
radius_pset4 = 3
num_bits_pset4 = 2048

class ECFP:
    def __init__(self, smiles):
        self.mols = [Chem.MolFromSmiles(i) for i in smiles]
        self.smiles = smiles

    def mol2fp(self, mol):
        bi = {}
        fp = AllChem.GetMorganFingerprintAsBitVect(mol,
                                                   radius = radius_pset4,
                                                   bitInfo = bi,
                                                   nBits = num_bits_pset4)
        array = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp, array)
        return array, bi

    def compute_ECFP(self):
        bit_headers = ['bit' + str(i) for i in range(num_bits_pset4)]
        arr = np.empty((0,num_bits_pset4), int).astype(int)
        bitInfo_all = []
        mol_all = []
        for i in self.mols:
            mol_all.append(i)
            fp, bi = self.mol2fp(i)
            arr = np.vstack((arr, fp))
            bitInfo_all.append(bi)
        df_fp = pd.DataFrame(np.asarray(arr).astype(int),columns=bit_headers)
        df_fp.insert(loc=0, column='smiles', value=self.smiles)
        df_fp.insert(loc=1, column='mol', value=mol_all)
        df_fp.insert(loc=2, column='bitInfo', value=bitInfo_all)
        return df_fp

smiles_standarized = [standardize_smiles(i) for i in jak2['SMILES'].values]
jak2_fp_descriptor = ECFP(smiles_standarized)
jak2_fp = jak2_fp_descriptor.compute_ECFP()
# Remove first column as we will reference smiles column from "jak2" dataframe
# Remove second and third columns because not needed for our exercise here
jak2_fp = jak2_fp.drop(columns=['smiles', 'mol', 'bitInfo'])
# This resulting dataframe, jak2_fp, contains the 2048 bits (columns) making up
# the fingerprints for the 1,911 molecules (rows)

### 1.1 (5 points, Grad students only) Choosing radius and number of bits for Morgan fingerprints


Provide a one-sentence description of what the radius represents and another of what the number of bits represents. How does adjusting the radius parameter affect the granularity of the motifs captured by the fingerprints, and how does this relate to the choice of the number of bits?

*INSERT YOUR ANSWER HERE*

### 1.2 (10 points) Principal Component Analysis on Molecular Fingerprints

Perform PCA to reduce data into vectors of 100 dimensions. Visualize the first two components of your data in  a 2D scatter plot and color each molecule by its pIC50.

In [ ]:
########## Modify this code chunk ##########
# Perform PCA


# Skeleton code for plotting
fig, ax = plt.subplots(figsize=(5, 5))
sc = ax.scatter(, , s=3, c=jak2['pIC50'], cmap='viridis')
cbar = plt.colorbar(sc)
cbar.set_label('pIC50', rotation=270, labelpad=15)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
plt.show()


What is the explained variance ratio of the 100 principal components?

In [ ]:
########## Insert your code in this chunk ##########


What patterns do you observe?

*INSERT YOUR ANSWER HERE*

### 1.3 (10 points) t-SNE analysis on Molecular Fingerprints

Perform t-SNE on the obtained principal components, with perplexity value of 2, 30, and 500. Plot the results and label your plots.

In [ ]:
########## Insert your code in this chunk ##########


What differences do you see between the 3 t-SNE plots? What patterns do you observe in the perplexity = 30 plot?

*INSERT YOUR ANSWER HERE*

### 1.4 (20 points, Grad students only) Are the low dimensional embeddings meaningful?

Discretize pIC50 data by classifying any molecule with a pIC50 value >= 9.5 as effective (i.e., 1) and any with <9.5 as ineffective (i.e., 0). Append this as a new column called `is_effective` to the `jak2` dataframe.

In [ ]:
########## Insert your code in this chunk ##########


Split the data into 10 folds. For each fold, train on the other 9 folds and validate on the last fold. Record your prediction.

In [ ]:
########## Insert your code in this chunk ##########


Classify your predictions into True Positives (TP), True Negatives (TN), False Positives (FP) and False Negatives (FN).

In [ ]:
########## Insert your code in this chunk ##########


Plot the 2D t-SNE embeddings (perplexity = 30) colored by the four classification classes.

In [ ]:
########## Insert your code in this chunk ##########


What pattern do you observe?

*INSERT YOUR ANSWER HERE*

________

## Part 2: Variational auto-encoders (VAEs) for proteins

In [ ]:
########## Simply run this chunk ##########
# Download data
paths = [
    "https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps4/data/bio_version/luxA_train.csv",
    "https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps4/data/bio_version/luxA_val.csv",
    "https://raw.githubusercontent.com/coleygroup/ML4MolEng/main/psets/ps4/data/bio_version/luxA_test.csv"
    ]

for path in paths:
    wget.download(path, "./")

In [ ]:
########## Simply run this chunk ##########
# Load data
luxA_train = pd.read_csv("luxA_train.csv")
luxA_val = pd.read_csv("luxA_val.csv")
luxA_test = pd.read_csv("luxA_test.csv")

In [ ]:
########## Simply run this chunk ##########
# Character list for FASTA protein sequences
aa_charset = ['-', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K',
              'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

# Define encoder
enc = preprocessing.LabelEncoder().fit(aa_charset)

# Determine max length of protein sequences
# Note that all have already been padded to have equivalent length
max_len = len(luxA_train.aa_sequence[0]) # 360

### 2.1 (5 points) Encode protein sequences into numerical vectors

Encode FASTA sequences into categorical vectors to make train/validation/test Datasets and DataLoaders.

In [ ]:
"########## Modify this code chunk ##########

X_train = None





X_val = None





X_test = None





train_data = None
train_loader = None

val_data = None
val_loader = None

test_data = None
test_loader = None


### 2.2 (15 points) Implement the reparametrization trick for VAE

In [ ]:
########## Modify this code chunk in the indicated places ##########
# Protein VAE model

class ProteinVAE(nn.Module):
    def __init__(self, rnn_enc_hid_dim, enc_nconv,
                 encoder_hid, z_dim,
                 rnn_dec_hid_dim, dec_nconv, aa_seq_len, nchar
                 ):
        '''
            Protein VAE model

                rnn_enc_hid_dim: hidden dimension for the GRU encoder
                enc_nconv: number of recurrent layers for the GRU decoder
                encoder_hid: dimension of GRU encoder readout
                z_dim: number of latent variable
                rnn_dec_hid_dim: hidden dimension for the GRU decoder
                dec_nconv: number of recurrent layers for the GRU decoder
                aa_seq_len: total length of aligned amino acid sequences
                nchar: number of possible characters

        '''

        super(ProteinVAE, self).__init__()

        self.aa_seq_len = aa_seq_len
        self.nchar = nchar
        # Embedding layer
        self.embed = nn.Embedding(self.nchar, rnn_enc_hid_dim)
        # Encoding GRU
        self.rnn_enc = nn.GRU(rnn_enc_hid_dim, rnn_enc_hid_dim, enc_nconv, batch_first=True)
        # MLP to transfrom hidden output from Encoding GRU
        self.mlp0 = nn.Linear(rnn_enc_hid_dim, encoder_hid)
        # Network to parametrize mu
        self.mu_network = nn.Linear(encoder_hid, z_dim)
        # Network to parametrize log variance
        self.logvar_network = nn.Linear(encoder_hid, z_dim)
        # Decoding GRU
        self.rnn_dec = nn.GRU(z_dim, rnn_dec_hid_dim, dec_nconv, batch_first=True)
        # Output SMILES characters
        self.readout = nn.Linear(rnn_dec_hid_dim, self.nchar)

    def encode(self, x):
        '''Output mean and log variance of the encoded SMILES'''
        output, hn = self.rnn_enc(x)
        h = torch.nn.functional.relu(self.mlp0(hn[-1]))
        return self.mu_network(h), self.logvar_network(h)

    def get_std(self, logvar):
        '''Transform log variance to standard deviation'''
        ################ Your code #################

    def reparametrize(self, mu, std):
        '''The reparametrization trick'''
        if self.training:
            ################ Your code #################


            return z
        else:
            return mu

    def decode(self, z):
        '''Decoder to reconstruct latent variable back to SMILES'''
        z = z.view(z.size(0), 1, z.size(-1)).repeat(1, self.aa_seq_len, 1)
        out, h = self.rnn_dec(z)
        out_reshape = out.contiguous().view(-1, out.size(-1))

        y0 = self.readout(out_reshape)
        y = y0.contiguous().view(out.size(0), -1, y0.size(-1))
        return y

    def forward(self, x):
        x_embed = self.embed(x) # Get SMILES embedding
        mu, logvar = self.encode(x_embed) # Encoding SMILES to latent representations
        std = self.get_std(logvar) # Transfrom log variance to std.
        z = self.reparametrize(mu, std) # Reparametrization trick
        smiles_recon = self.decode(z)  # Reconstruct SMILES string
        return smiles_recon, mu, std

Test your model by comparing your sampling with N(0, 1).

In [ ]:
########## Simply run this chunk ##########
# Define your model
model = ProteinVAE(rnn_enc_hid_dim=256, enc_nconv=1,
                   encoder_hid=256, z_dim=128, rnn_dec_hid_dim=512,
                   dec_nconv=3, nchar=len(aa_charset), aa_seq_len=max_len)

# Compare your sampling with N(0, 1)

import matplotlib.pyplot as plt
from scipy.stats import norm

sample = model.reparametrize(torch.zeros(1000), torch.ones(1000))
plt.hist(sample.detach().cpu().numpy(), density=True)

# Plot between -10 and 10 with .001 steps.
x_axis = np.arange(-7, 7, 0.001)
plt.plot(x_axis, norm.pdf(x_axis,0,1)) # Mean = 0, SD = 1.
plt.show()

### 2.3 (10 points) Implement the protein VAE loss function

Implement your loss function here.

In [ ]:
########## Modify this code chunk ##########
def loss_function(recon_x, x, mu, std):
    BCE =
    KLD =
    return BCE, KLD


### 2.4 (5 points) Train your model

Simply run the following code chunks to train your model.

In [ ]:
########## Simply run this chunk ##########
def loop(model, loader, epoch, beta=0.05, evaluation=False):
    '''
        Train/test your VAE model
    '''

    if evaluation:
        model.eval()
        mode = "eval"
    else:
        model.train()
        mode = 'train'
    batch_losses = []

    tqdm_data = tqdm(loader, position=0, leave=True, desc='{} (epoch #{})'.format(mode, epoch))
    for data in tqdm_data:

        x = data[0].to(device)
        recon_batch, mu, std = model(x)
        loss_recon, loss_kl = loss_function(recon_batch, x, mu, std)
        loss = loss_recon + beta * loss_kl

        if not evaluation:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        batch_losses.append(loss.item())

        postfix = ['recon loss={:.3f}'.format(loss_recon.item()) ,
                   'KL loss={:.3f}'.format(loss_kl.item()) ,
                   'total loss={:.3f}'.format(loss.item()) ,
                   'avg. loss={:.3f}'.format(np.array(batch_losses).mean())]

        tqdm_data.set_postfix_str(' '.join(postfix))

    return np.array(batch_losses).mean()

In [ ]:
########## Simply run this chunk ##########
device = 0

model = ProteinVAE(rnn_enc_hid_dim=256, enc_nconv=1,
                   encoder_hid=256, z_dim=128, rnn_dec_hid_dim=512,
                   dec_nconv=3, nchar=len(aa_charset), aa_seq_len=max_len)

model = model.to(device)

In [ ]:
########## Simply run this chunk ##########
optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

Optional: mount your Google Drive to save your model and files.

In [ ]:
########## Optional: simply run this chunk ##########
# Optional: mount your google drive to save model and files
# Uncomment below lines if of interest

#from google.colab import drive
#drive.mount('/content/drive')
#mydrive = '/content/drive/MyDrive'

In [ ]:
########## Simply run this chunk ##########
# Takes 35 minutes
epochs = 50

for epoch in range(0, epochs):

    # sample_recon(model, epoch, val_loader, enc)
    train_loss = loop(model, train_loader, epoch, 0.001)
    val_loss = loop(model, val_loader, epoch, 0.001,  evaluation=True)
    scheduler.step(val_loss)

    # optional: save model
    # uncoment to save model if of interest
    #if epoch % 15 == 0:
    #     torch.save(model.state_dict(),
    #             '{}/vae-{:03d}-{:.2f}.pth'.format(mydrive, epoch, train_loss))

    #     torch.save(optimizer.state_dict(),
    #         '{}/optim-{:03d}-{:.2f}.pth'.format(mydrive, epoch, train_loss))

    if epoch == 0:
        best_loss = train_loss.item()
    else:
        if train_loss.item() < best_loss:
            best_loss = train_loss.item()

### 2.5 (20 points) Sample new protein sequences

A helper function for you.

In [ ]:
########## Simply run this chunk ##########
# Helper function
def index2fasta(mol_index, enc):
    '''Transform your array of character indices back to FASTA'''
    fasta_charlist = enc.inverse_transform(np.array(mol_index))
    fasta = ''.join(fasta_charlist).strip(" ")

    return fasta

Randomly select two FASTA sequences in your test data, interpolate 10 points between them, and decode those points.

In [ ]:
########## Modify this code chunk ##########
# select a starting and ending protein sequence

start = index2fasta(test_loader.dataset.__getitem__(random.choices(range(len(test_loader.dataset)), k=1))[0].numpy().reshape(-1), enc)
end = index2fasta(test_loader.dataset.__getitem__(random.choices(range(len(test_loader.dataset)), k=1))[0].numpy().reshape(-1), enc)

model.eval()

################ Your code #################



Produce a scatter plot with the first two dimensions of $z$ of your test molecules and newly sampled molecules in the same figure. Color differently the test points and generated points.

In [ ]:
########## Insert your code in this chunk ##########


In a separate table, print the 10 different FASTA sequences you generated.

In [ ]:
########## Insert your code in this chunk ##########


Briefly comment on the sequence diversity you observe among these 10 generated sequences. How many are essentially identical? What do you think can be done to improve the model in its ability to finely interpolate sequences in this learned latent space?

*INSERT YOUR ANSWER HERE*